In [38]:
using Revise
using Distributions, LinearAlgebra, StatsBase
using Test

In [39]:
includet("msvar_temp.jl")


In [40]:
St = [.9, .1]
St1 = [.8, .2]
St1T = [.85, .15]

transition_matrix = [.9 .1;.2 .8]

r = transition_matrix * (St1T ./ St1) .* St



2-element Vector{Float64}:
 0.928125
 0.08125

In [41]:
n_samples = 10_000
regimes = sample([1,2], ProbabilityWeights([0.8, 0.2]), n_samples)

dist1 = Normal(.13,.08)
dist2 = Normal(-.03, .2)


samples = zeros(n_samples)

for s in 1:n_samples
    if regimes[s] == 1
        samples[s] = rand(dist1)
    else 
        samples[s] = rand(dist2)
    end
end

display(mean(samples))
display(std(samples))
display(skewness(samples))
display(kurtosis(samples))



0.09869087727942837

0.1312189436489359

-1.260538366868947

3.452835806344594

In [42]:
mu = [0.8, 0.2]' * [.13, -.03]
sig = sqrt([0.8, 0.2]' * [.08^2, .2^2])

samples = rand(Normal(mu, sig), n_samples)

display(mean(samples))
display(std(samples))
display(skewness(samples))
display(kurtosis(samples))

0.09577983513731654

0.11397424644853196

0.03912169301243186

0.01382103721418293

In [43]:
var = .8(1-.8)*(.13+.03)^2 + .8 * .08^2 + (1-.8) * .2^2

sqrt(var)

0.1312097557348538

Simulate MSVAR TODO 
- first period simulate regimes
- simulate returns for every scenarios
- for next periods:
  - simulate regime based on current regime and transition matrix
  - siumlate returns based on regime

Two regimes, tree regimes, constand means, coeff, sigmas.



In [64]:
X = [.12, .06]

transition_matrix = [.9 .1;.2 .8]

states_zero = [.67, .33]


Β1 = [.08 .3 .2; .03 .1 .6]
Σ1 = cor2cov([1 .3; .3 1], [.15, .03])
Β2 = [.085 .35 .25; .035 .15 .63]
Σ2 = cor2cov([1 .35; .35 1], [.16, .035])
 

regimes = simulate_next_regimes(states_zero, transition_matrix, 2_000)
display(regimes)
result = simulate_returns_step(regimes, [Β1, Β2], [Σ1, Σ2], repeat(X, 1, 2_000))

@test mean(result, dims=2) ≈ [.132, .081] atol=.009



2000-element Vector{Int64}:
 2
 1
 1
 1
 2
 2
 2
 1
 1
 1
 ⋮
 1
 1
 1
 1
 2
 1
 2
 2
 1

Test Passed

In [45]:
n_scenarios = 10

X1 = add_intercept(repeat(X, 1, n_scenarios), n_scenarios)

3×10 Matrix{Float64}:
 1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0
 0.12  0.12  0.12  0.12  0.12  0.12  0.12  0.12  0.12  0.12
 0.06  0.06  0.06  0.06  0.06  0.06  0.06  0.06  0.06  0.06

In [46]:
repeat(X, 1, n_scenarios)

2×10 Matrix{Float64}:
 0.12  0.12  0.12  0.12  0.12  0.12  0.12  0.12  0.12  0.12
 0.06  0.06  0.06  0.06  0.06  0.06  0.06  0.06  0.06  0.06

In [47]:
result1 = simulate_msvar(transition_matrix, [Β1, Β1], [Σ1, Σ2], X, states_zero, 3, 2_000)
@test mean(result1[:,1,:], dims=2) ≈ [.132, .081] atol=.009


Test Passed

In [48]:
display(mean(result1[1,1,:]))
display(std(result1[1,1,:]))
display(skewness(result1[1,1,:]))
display(kurtosis(result1[1,1,:]))

0.12656290820042565

0.1527488750163373

-0.18914630262819765

-0.0031198403721912626

In [49]:
regimes2 = simulate_next_regimes_for_previous_regimes(regimes, transition_matrix)

expected_n_regime_one = (0.7 * 0.9 + 0.3 * 0.2) * 2000
@test count(i -> i == 1, regimes2 ) ≈ 1380 atol=80

Test Passed

In [50]:
result = simulate_msvar(transition_matrix, [Β1, Β1], [Σ1, Σ2], X, states_zero, 3, 2_000)

mean(result[:,2,:],dims=2)

2×1 Matrix{Float64}:
 0.13500469027033124
 0.08992705844880507

In [51]:
result = simulate_msvar(transition_matrix, [Β1, Β1], [Σ1, Σ2], X, states_zero, 3, 2_000)

mean(result[:,3,:],dims=2)

2×1 Matrix{Float64}:
 0.13548922946663902
 0.09818230279791298

In [52]:
X = [.12, .06]

transition_matrix = [.9 .1;.2 .8]

states_zero = [.67, .33]


Β1 = [.09 .3 .2; .03 .1 .6]
Σ1 = cor2cov([1 .3; .3 1], [.08, .03])
Β2 = [-.02 .35 .25; .035 .15 .63]
Σ2 = cor2cov([1 .35; .35 1], [.18, .035])

result = simulate_msvar(transition_matrix, [Β1, Β1], [Σ1, Σ2], X, states_zero, 120, 100_000)

display("T1")
display(mean(result[1,1,:]))
display(std(result[1,1,:]))
display(skewness(result[1,1,:]))
display(kurtosis(result[1,1,:]))

display("T10")
display(mean(result[1,10,:]))
display(std(result[1,10,:]))
display(skewness(result[1,10,:]))
display(kurtosis(result[1,10,:]))

"T1"

0.13831945157705094

0.12225820431026627

-0.0022017047378315446

2.0366475587028665

"T10"

0.1604135506836415

0.1301724704221378

-0.0008298468601871306

1.8470822840766736

expected mean 
for regime 1 
prob regime one, previous regime one mean, 
prob regime 2 to 1, previous regime two mean

In [53]:
0.669 * 300 

200.70000000000002

In [54]:
Β1 = [.08 .3 .2; .03 .1 .6]'
X = [1. .12 .06]

X * Β1

1×2 Matrix{Float64}:
 0.128  0.078

In [55]:
Β2 = [.085 .35 .25; .035 .15 .63]'
Σ2 = cor2cov([1 .35; .35 1], [.16, .035])

X * Β2

1×2 Matrix{Float64}:
 0.142  0.0908

In [56]:
.7 * .128 + .3 * .142 

0.13219999999999998

In [57]:
.7 * .078 + .3 * .0908

0.08184

In [58]:
.08 * 1 + .3 * .12 + .2 * .06

0.128

In [59]:
mean([.1 .2; .3 .4], dims=2)

2×1 Matrix{Float64}:
 0.15000000000000002
 0.35

In [60]:
Σ1

2×2 Matrix{Float64}:
 0.0064   0.00072
 0.00072  0.0009

In [61]:
cholesky(Σ1).L 

2×2 LowerTriangular{Float64, Matrix{Float64}}:
 0.08    ⋅ 
 0.009  0.0286182

In [62]:
using SimpleMock, Distributions

mock(rand => Mock(1)) do mockrand
    rand(1:2) == 1
    rand(Normal(0,2)) 
    called_with(mockrand, Normal(0,1))
end


ArgumentError: ArgumentError: Package SimpleMock not found in current path, maybe you meant `import/using .SimpleMock`.
- Otherwise, run `import Pkg; Pkg.add("SimpleMock")` to install the SimpleMock package.

In [76]:
.635 * 300 

190.5

In [77]:
transition_matrix = [.85 .15;.2 .8]
r1 = transition_matrix' * states_zero

2-element Vector{Float64}:
 0.6355000000000001
 0.3645

In [83]:
r2 = transition_matrix' * r1
display(r2)
r2 * 300 

2-element Vector{Float64}:
 0.613075
 0.386925

2-element Vector{Float64}:
 183.9225
 116.0775

In [73]:
0.636 * 300

190.8

In [90]:
transition_matrix

2×2 Matrix{Float64}:
 0.85  0.15
 0.2   0.8

In [104]:
r1  = sample([1,2], ProbabilityWeights(transition_matrix[1,:]),10_000)
count(i -> i==1, r1)

8477

In [149]:
rand(Multinomial(1000, [0.85, 0.15]))

2-element Vector{Int64}:
 844
 156

In [144]:
transition_matrix

2×2 Matrix{Float64}:
 0.85  0.15
 0.2   0.8

In [145]:
states_zero

2-element Vector{Float64}:
 0.67
 0.33

In [148]:
.67 * .85 + .33 * .2

0.6355

In [150]:
rand(1000)

1000-element Vector{Float64}:
 0.32499698583498904
 0.660982165391218
 0.13836928847637464
 0.32412470405698723
 0.28267391076757464
 0.08538186201125919
 0.37832573921206114
 0.9024006322052698
 0.7567319491565474
 0.9981730321916807
 ⋮
 0.114603734035859
 0.5013434304486624
 0.4411966898229459
 0.7909464943581676
 0.6855369530698091
 0.8221709222536435
 0.9612315393783853
 0.27905139569727366
 0.2810082662828951

In [155]:
include("../src/ESGModels/ESGModels.jl")

Main.ESGModels

In [175]:
X = [1, .12, .06]

transition_matrix = [.85 .15;.2 .8]

states_zero = [.67, .33]


Β1 = [.09 .3 .2; .03 .1 .6]
Σ1 = cor2cov([1 .3; .3 1], [.08, .03])
Β2 = [-.02 .35 .25; .035 .15 .63]
Σ2 = cor2cov([1 .35; .35 1], [.18, .035])

using .ESGModels

er= ESGModels.expected_regimes(states_zero, transition_matrix, 3)
display(er)
expected_returns_t1 = ESGModels.predict(Β1, X) * er[1,1] + predict(Β2,X) * er[1,2]
display(expected_returns_t1)

X2 = ESGModels.add_intercept(expected_returns_t1, 1)[:,1]

expected_returns_t2_regime1 = ESGModels.predict(Β1, X2) * er[2,1] + predict(Β2,X2) * er[2,2]

#t2_r1_returns = t1_returns_r1 * t11 + t1_returns_r2 * t21
#t2_r2_returns = t1_returns_r1 * t12 + t1_returns_r2 * t22
#returns = ???
#t2r1 = r1 * t11 + r2 * t21 

3×2 Matrix{Float64}:
 0.6355    0.3645
 0.613075  0.386925
 0.598499  0.401501

2-element Vector{Float64}:
 0.10118550000000001
 0.0826656

2-element Vector{Float64}:
 0.097883849343375
 0.09456965659777503

In [186]:
initial_states = [(transition_matrix[2,1]/(transition_matrix[2,1]+ transition_matrix[1,2])), transition_matrix[1,2]/(transition_matrix[2,1]+ transition_matrix[1,2])]


2-element Vector{Float64}:
 0.5714285714285715
 0.4285714285714286

In [178]:
v = [1, 2]
display(size(v))
m = [1 2]
display(size(m))

(2,)

(1, 2)